In [19]:
import pandas as pd 
import numpy as np 
import sys
from sklearn.preprocessing import MinMaxScaler 
from autogluon.tabular import TabularDataset, TabularPredictor
import warnings
from fastprogress import fastprogress
warnings.filterwarnings("ignore")

#Load a tabular & test dataset:
train_data = TabularDataset('../dataset/CIC-2018/train_ori.csv')
test_data = TabularDataset('../dataset/CIC-2018/test.csv')

print(train_data['Label'].value_counts())
print(test_data['Label'].value_counts())

Loaded data from: ../dataset/CIC-2018/train_ori.csv | Columns = 80 / 80 | Rows = 99889 -> 99889
Loaded data from: ../dataset/CIC-2018/test.csv | Columns = 80 / 80 | Rows = 42540 -> 42540


Bot                         14084
Infilteration               14048
DoS attacks-GoldenEye       14046
Benign                      14001
DoS attacks-Hulk            13969
DDOS attack-HOIC            13947
DoS attacks-SlowHTTPTest     9541
DoS attacks-Slowloris        5106
DDOS attack-LOIC-UDP          875
Brute Force -Web              175
Brute Force -XSS               77
SQL Injection                  20
Name: Label, dtype: int64
DDOS attack-HOIC            6053
DoS attacks-Hulk            6031
Benign                      5999
DoS attacks-GoldenEye       5954
Infilteration               5952
Bot                         5916
DoS attacks-SlowHTTPTest    4082
DoS attacks-Slowloris       2093
DDOS attack-LOIC-UDP         336
Brute Force -Web              78
Brute Force -XSS              29
SQL Injection                 17
Name: Label, dtype: int64


In [20]:

print("Fit classification models predicting:")
predictor = TabularPredictor(label='Label').fit(train_data)  # Fit models for 12000s # time_limit=12000, presets='best_quality'

print("Evaluate predictions on test data:")
leaderboard = predictor.leaderboard(test_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20220911_133622/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220911_133622/"
AutoGluon Version:  0.5.0
Python Version:     3.8.10
Operating System:   Linux
Train Data Rows:    99889
Train Data Columns: 79
Label Column: Label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 12) unique label values:  ['DoS attacks-SlowHTTPTest', 'DoS attacks-Slowloris', 'DDOS attack-LOIC-UDP', 'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection', 'Benign', 'Bot', 'DDOS attack-HOIC', 'DoS attacks-Hulk']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during

Fit classification models predicting:


Train Data Class Count: 12
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    9723.15 MB
	Train Data (Original)  Memory Usage: 63.13 MB (0.6% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 10 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Useless Original Features (Count: 8): ['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature w

Evaluate predictions on test data:
                  model  score_test  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      RandomForestEntr    0.999154     0.9984        1.569046       0.178078    16.631284                 1.569046                0.178078          16.631284            1       True          7
1        ExtraTreesGini    0.999107     0.9996        1.699444       0.213217     6.567382                 1.699444                0.213217           6.567382            1       True          9
2         LightGBMLarge    0.999060     0.9992        0.171581       0.015520   434.788489                 0.171581                0.015520         434.788489            1       True         13
3      RandomForestGini    0.998989     0.9988        1.636108       0.229307    13.353849                 1.636108                0.229307          13.353849            1       True       

In [25]:
print("The best model:")
print(predictor.get_model_best())
print("------------------------------------------------")
print(predictor.evaluate(test_data))

The best model:
WeightedEnsemble_L2
------------------------------------------------


Evaluation: accuracy on test data: 0.9980253878702398
Evaluations on test data:
{
    "accuracy": 0.9980253878702398,
    "balanced_accuracy": 0.9296665667449773,
    "mcc": 0.9977292595210285
}


{'accuracy': 0.9980253878702398, 'balanced_accuracy': 0.9296665667449773, 'mcc': 0.9977292595210285}


In [26]:
metric=['accuracy', 'precision_weighted', 'f1_weighted', 'recall_weighted']
predictor.leaderboard(test_data, extra_metrics=metric, silent=True)

,model,score_test,accuracy,precision_weighted,f1_weighted,recall_weighted,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.999154,0.999154,0.999166,0.999146,0.999154,0.9984,1.406620,0.178078,16.631284,1.406620,0.178078,16.631284,1,True,7
1,ExtraTreesGini,0.999107,0.999107,0.999113,0.999105,0.999107,0.9996,1.804368,0.213217,6.567382,1.804368,0.213217,6.567382,1,True,9
2,LightGBMLarge,0.999060,0.999060,0.999068,0.999062,0.999060,0.9992,0.182849,0.015520,434.788489,0.182849,0.015520,434.788489,1,True,13
3,RandomForestGini,0.998989,0.998989,0.999002,0.998982,0.998989,0.9988,1.440966,0.229307,13.353849,1.440966,0.229307,13.353849,1,True,6
4,ExtraTreesEntr,0.998942,0.998942,0.998950,0.998940,0.998942,0.9996,1.457038,0.207132,6.557843,1.457038,0.207132,6.557843,1,True,10
5,KNeighborsDist,0.998684,0.998684,0.998699,0.998672,0.998684,1.0000,303.051278,8.442643,0.283316,303.051278,8.442643,0.283316,1,True,2
6,CatBoost,0.998519,0.998519,0.998550,0.998464,0.998519,1.0000,0.276238,0.015108,186.450942,0.276238,0.015108,186.450942,1,True,8
7,XGBoost,0.998449,0.998449,0.998452,0.998444,0.998449,0.9992,0.544903,0.028779,26.266391,0.544903,0.028779,26.266391,1,True,11
8,KNeighborsUnif,0.998025,0.998025,0.998043,0.997955,0.998025,1.0000,318.467911,10.131547,0.281579,318.467911,10.131547,0.281579,1,True,1
9,WeightedEnsemble_L2,0.998025,0.998025,0.998043,0.997955,0.998025,1.0000,318.473987,10.132526,1.794800,0.006076,0.000979,1.513221,2,True,14
